#  **Redes Quânticas - Exemplo 2 - A primeira rede quântica**

Neste notebook agora combinamos todas as partes e criamos uma rede que consiste em dois nós, uma conexão com um canal quântico e uma conexão com um canal clássico. Em seguida, executamos um protocolo completo de teletransporte.


In [1]:
import netsquid as ns
import pandas
from netsquid.components.qprocessor import QuantumProcessor, PhysicalInstruction
from netsquid.nodes import Node, Connection, Network
from netsquid.protocols.protocol import Signals
from netsquid.protocols.nodeprotocols import NodeProtocol
from netsquid.components.qchannel import QuantumChannel
from netsquid.components.cchannel import ClassicalChannel
from netsquid.qubits.state_sampler import StateSampler
from netsquid.components.qprogram import QuantumProgram
from netsquid.components.models.qerrormodels import DepolarNoiseModel, DephaseNoiseModel
from netsquid.components.models.delaymodels import FibreDelayModel, FixedDelayModel
import netsquid.components.instructions as instr
import pydynaa

<br><br>Vamos seguir as seguintes etapas:
- primeiro, inicializamos três qubits no node2a;
- mudamos o estado do primeiro qubit para ..;
- então emaranhamos os outros dois qubits;
- enviamos um dos qubits emaranhados para node2b;
- agora realizamos um BSM nos dois qubits restantes no node2a;
- enviamos o resultado de ambas as medições para node2b;
- corrigimos o qubit no node2b;
- verificamos a fidelidade do estado teletransportado.

Primeiro, definimos como sempre as instruções físicas e os processadores quânticos.

In [2]:
phys_instructions2 = [
    PhysicalInstruction(instr.INSTR_INIT, duration=3),
    PhysicalInstruction(instr.INSTR_S, duration=1, parallel = True, topology =[0]),
    PhysicalInstruction(instr.INSTR_H, duration=1, prallel = True, topology = [0,1]),
    PhysicalInstruction(instr.INSTR_X, duration=1, parallel = True, topology = [0]),
    PhysicalInstruction(instr.INSTR_Z, duration=1, parallel = True, topology = [0]),
    PhysicalInstruction(instr.INSTR_CNOT, duration=1, parallel = True, topology = [(0,1),(1,2)]),
    PhysicalInstruction(instr.INSTR_MEASURE, duration=7, parallel = True, topology = [0,1])
]

qproc2a = QuantumProcessor(name="ExampleQProc2a", num_positions = 3, phys_instructions = phys_instructions2)
qproc2b = QuantumProcessor(name="ExampleQProc2b", num_positions = 3, phys_instructions = phys_instructions2)

<br><br> <span style = "color: orange"> class netsquid.nodes.connections.Connection(name)</span>

In [3]:
class QuantumConnection(Connection):
    """Uma conexão neste exemplo contém apenas um canal quântico.
No entanto, podemos incluir uma fonte quântica para garantir a conexão 
e usá-la para distribuir qubits emaranhados aos nós conectados."""
    def __init__(self, name="QuantumConnection"):
        super().__init__(name=name)

        # Inicializando o canal quântico
        qchannel1 = QuantumChannel(name="QChannel_2a22b")

        #Conectando o canal às portas de entrada e saída da conexão
        self.add_subcomponent(qchannel1,
                             forward_input=[("A","send")],
                             forward_output=[("B","recv")])

In [4]:
class ClassicalConnection(Connection):
    """Uma conexão com um canal clássico"""
    def __init__(self, name="ClassicalConnection"):
        super().__init__(name=name)

        # Inicializando o canal clássico
        cchannel1 = ClassicalChannel(name="QChannel_2a22b")

        #Conectando o canal às portas de entrada e saída da conexão
        self.add_subcomponent(cchannel1,
            forward_input=[("A","send")],
            forward_output=[("B","recv")])

<br><br><span style = "color:orange"> class netsquid.nodes.network.Network(name, nodes = None) </span>

In [5]:
def network_setup2():
    """Configurando todos os componentes da rede"""
    
    # Primeiro configuramos os dois nós com seus processadores
    node2a = Node(name="ExampleNode2a", qmemory = qproc2a, port_names=['cin_2a','cout_2a','qin_2a','qout_2a'])
    node2b = Node(name="ExampleNode2b", qmemory = qproc2b, port_names=['cin_2b','cout_2b','qin_2b','qout_2b'])
    
    # Em seguida, nós criamos a rede
    network2 = Network("ExampleNet2")
    
    #Agora, nós adicionamos dois nós para a rede
    network2.add_nodes([node2a, node2b])
    
    # Configurando a conexão clássica entre os dois nós
    cconn = ClassicalConnection()
    
   # E  adicionando as conexões na rede
    network2.add_connection(node2a, node2b, connection = cconn, label = "classical connection", 
                            port_name_node1="cout_2a", port_name_node2="cin_2b")
    # configurandoa conexão quântica entre dois nós
    qconn = QuantumConnection()
    network2.add_connection(node2a, node2b, connection = qconn, label = "quantum connection", 
                            port_name_node1="qout_2a", port_name_node2="qin_2b")
    
    # por último, adicionamos conexões as portas dentro de cada nó
    node2a.qmemory.ports['qout2'].forward_output(node2a.ports['qout_2a'])
    node2b.ports['qin_2b'].forward_input(node2b.qmemory.ports['qin0'])
    
    return network2

In [6]:
class node2aProgram1(QuantumProgram):
    default_num_qubits = 3 # número de qubits
    
    def program(self):
        q0, q1, q2 = self.get_qubit_indices(3)
        self.apply(instr.INSTR_INIT, [q0,q1,q2]) # inicializando os três qubits
        
        # Configurando o estado do qubit 1 para y0
        self.apply(instr.INSTR_H, [q0]) # Aplicando a porta H em q0
        self.apply(instr.INSTR_S, [q0]) # Aplicando a porta S em q0
        
        # Emaranhando os qubits 2 e 3
        self.apply(instr.INSTR_H, [q1]) # Aplicando a porta H em q1
        self.apply(instr.INSTR_CNOT, [q1,q2]) # Aplicando a porta CNOT em q1 como controle e q2 como alvo
        
        yield self.run() # Rodando as instruções acima
    
class node2aProgram2(QuantumProgram):
    default_num_qubits = 2 # Número de qubits
    
    def program(self):
        q0, q1 = self.get_qubit_indices(2)
        
        # Agora emaranhando os qubits 1 e 2 e medindo ambos 
        self.apply(instr.INSTR_CNOT, [q0,q1]) # Aplicando a porta CNOT em q0 como controle e q1 como alvo
        self.apply(instr.INSTR_H, [q0]) # Aplicando a porta H em q0
        self.apply(instr.INSTR_MEASURE, q0, output_key = "M1") # medir o q0 e armazena o resultado
        self.apply(instr.INSTR_MEASURE, q1, output_key = "M2") # medir o q1 e armazena o resultado
        
        yield self.run() # Rodando as instruções agendadas acima

In [7]:
class node2aProtocol(NodeProtocol):
    def run(self):
        node2aprog1 = node2aProgram1()
        node2aprog2 = node2aProgram2()
        
        yield self.node.qmemory.execute_program(node2aprog1)
        
        qubit = self.node.qmemory.pop(positions=[2])
        
        self.node.qmemory.ports["qout2"].tx_output(qubit)
        
        yield self.node.qmemory.execute_program(node2aprog2)
        
        m1, = node2aprog2.output["M1"]
        m2, = node2aprog2.output["M2"]
        print("measurement outcome", m1, m2)
        self.node.ports["cout_2a"].tx_output((m1,m2))
        
        
class node2bProtocol(NodeProtocol):
    def run(self):
        port_cin = self.node.ports["cin_2b"]
        port_qin = self.node.ports["qin_2b"]
        
        # Aguardando o qubit chegar aqui
        expr = yield(self.await_port_input(port_qin))
        
        
        # Aguardando a mensagem chegar aqui
        expr = yield(self.await_port_input(port_cin))
        message, = self.node.ports["cin_2b"].rx_input().items
        print("message", message)
        if message[0] == 1:
            self.node.qmemory.execute_instruction(instr.INSTR_X,[0]) # Aplicando a porta H
            print("aplicando porta X")
            yield self.await_program(self.node.qmemory)
        
        if message[1] == 1:
            self.node.qmemory.execute_instruction(instr.INSTR_Z,[0]) # Aplicando a porta Z
            print("aplicando porta Z")
            yield self.await_program(self.node.qmemory)

In [8]:
network = network_setup2()

node2a = network.get_node("ExampleNode2a")
node2b = network.get_node("ExampleNode2b")

node2aprot = node2aProtocol(node2a)
node2bprot = node2bProtocol(node2b)

node2aprot.start() # iniciando o protocolo no node2a
node2bprot.start() # iniciando o protocolo no node2b

ns.sim_run() # Inicia a simulação
ns.sim_time

measurement outcome 0 0
message (0, 0)


<function netsquid.util.simtools.sim_time(magnitude=1)>

In [9]:
node2b.qmemory.peek(positions=[0])

[Qubit('QS#0-2')]

In [10]:
qubit, = node2b.qmemory.peek(positions=[0])
print("Fidelity of teleported state:{}".format(ns.qubits.fidelity([qubit],ns.y0)))

Fidelity of teleported state:0.9999999999999998
